In [1]:
import os
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
site = pd.read_csv('data/site_info.csv')
energy = pd.read_csv('data/energy.csv')
dangjin_fcst = pd.read_csv('data/add_dangjin_wind_time_insolation.csv')
ulsan_fcst = pd.read_csv('data/add_ulsan_wind_time_insolation.csv')

In [3]:
energy

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00:00,0.0,0.0,0,0
1,2018-03-01 2:00:00,0.0,0.0,0,0
2,2018-03-01 3:00:00,0.0,0.0,0,0
3,2018-03-01 4:00:00,0.0,0.0,0,0
4,2018-03-01 5:00:00,0.0,0.0,0,0
...,...,...,...,...,...
25627,2021-01-31 20:00:00,0.0,0.0,0,0
25628,2021-01-31 21:00:00,0.0,0.0,0,0
25629,2021-01-31 22:00:00,0.0,0.0,0,0
25630,2021-01-31 23:00:00,0.0,0.0,0,0


In [4]:
dangjin_fcst

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0


In [5]:
site

,Id,Capacity,Address,InstallationAngle,IncidentAngle,Latitude,Longitude
0,당진수상태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
1,당진자재창고태양광,0.7,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
2,당진태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
3,울산태양광,0.5,울산광역시 남구 용잠로 623,20.0,20.0,35.477651,129.380778


In [6]:
c_floating = 1000
c_warehouse = 700
c_dangjin = 1000
c_ulsan = 500

In [7]:
model_path = 'model'
filename = []
filename.append(os.path.join(model_path, 'floating_checkpoint.h5py'))
filename.append(os.path.join(model_path, 'warehouse_checkpoint.h5py'))
filename.append(os.path.join(model_path, 'dangjin_checkpoint.h5py'))
filename.append(os.path.join(model_path, 'ulsan_checkpoint.h5py'))

In [8]:
def make_dataset(data, label, window_size=24):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [9]:
def root_mean_squared_error (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [10]:
df = pd.concat([dangjin_fcst, energy['dangjin_floating']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,dangjin_floating
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [11]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['dangjin_floating'] / c_floating
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [12]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [13]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16796, 24, 10), (4199, 24, 10), (4541, 24, 10), (672, 24, 10))

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM
from tensorflow.keras import backend as K

epoch = 50

In [15]:
model_0 = Sequential()
model_0.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_0.add(Dropout(0.3))
model_0.add(Dense(1, activation='sigmoid'))
model_0.compile(loss='mean_squared_error', optimizer='adam')
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 24, 24)            3360      
_________________________________________________________________
dropout (Dropout)            (None, 24, 24)            0         
_________________________________________________________________
dense (Dense)                (None, 24, 1)             25        
Total params: 3,385
Trainable params: 3,385
Non-trainable params: 0
_________________________________________________________________


In [16]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[0], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_0 = model_0.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
522/525 [============================>.] - ETA: 0s - loss: 0.0358
Epoch 00001: val_loss improved from inf to 0.01627, saving model to model\floating_checkpoint.h5py
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
525/525 [==============================] - 6s 11ms/step - loss: 0.0357 - val_loss: 0.0163
Epoch 2/50
522/525 [============================>.] - ETA: 0s - loss: 0.0147
Epoch 00002: val_loss improved from 0.01627 to 0.01277, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
525/525 [==============================] - 5s 10ms/step - loss: 0.0146 - val_loss: 0.0128
Epoch 3/50
523/525 [============================>.] - ETA: 0s - loss: 0.0119
Epoch 0

Epoch 25/50
523/525 [============================>.] - ETA: 0s - loss: 0.0065
Epoch 00025: val_loss did not improve from 0.00614
525/525 [==============================] - 4s 7ms/step - loss: 0.0065 - val_loss: 0.0063
Epoch 26/50
524/525 [============================>.] - ETA: 0s - loss: 0.0065
Epoch 00026: val_loss did not improve from 0.00614
525/525 [==============================] - 4s 7ms/step - loss: 0.0065 - val_loss: 0.0062
Epoch 27/50
523/525 [============================>.] - ETA: 0s - loss: 0.0065
Epoch 00027: val_loss improved from 0.00614 to 0.00605, saving model to model\floating_checkpoint.h5py
INFO:tensorflow:Assets written to: model\floating_checkpoint.h5py\assets
525/525 [==============================] - 5s 10ms/step - loss: 0.0065 - val_loss: 0.0061
Epoch 28/50
519/525 [============================>.] - ETA: 0s - loss: 0.0065
Epoch 00028: val_loss did not improve from 0.00605
525/525 [==============================] - 4s 7ms/step - loss: 0.0065 - val_loss: 0.0064
Ep

In [23]:
floating_pred = model_0.predict(pred_feature)
floating_pred

array([[8.42552618e-05],
       [8.43778907e-05],
       [9.42818151e-05],
       [1.26242638e-04],
       [2.33381987e-04],
       [5.52535057e-04],
       [1.87751651e-03],
       [8.62383842e-03],
       [4.74213660e-02],
       [1.03984922e-01],
       [1.86048925e-01],
       [2.61934698e-01],
       [3.33082318e-01],
       [3.64852875e-01],
       [3.45188081e-01],
       [2.89075255e-01],
       [1.58655971e-01],
       [4.52697277e-02],
       [1.77457929e-02],
       [3.91927361e-03],
       [6.58184290e-04],
       [1.94638968e-04],
       [1.07088264e-04],
       [7.89756450e-05],
       [7.45044235e-05],
       [8.04646625e-05],
       [1.01415244e-04],
       [1.48147345e-04],
       [2.88337469e-04],
       [8.51690769e-04],
       [3.45429778e-03],
       [2.37044096e-02],
       [1.30512089e-01],
       [2.68919617e-01],
       [3.77836883e-01],
       [4.25768077e-01],
       [4.38470274e-01],
       [4.23310578e-01],
       [3.72735918e-01],
       [2.99967378e-01],


In [17]:
df = pd.concat([dangjin_fcst, energy['dangjin_warehouse']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,dangjin_warehouse
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [18]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['dangjin_warehouse'] / c_warehouse
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [19]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [20]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16776, 24, 10), (4195, 24, 10), (4541, 24, 10), (672, 24, 10))

In [25]:
model_1 = Sequential()
model_1.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_1.add(Dropout(0.1))
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(loss='mean_squared_error', optimizer='adam')
model_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 24, 24)            3360      
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 24)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 24, 1)             25        
Total params: 3,385
Trainable params: 3,385
Non-trainable params: 0
_________________________________________________________________


In [26]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[1], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_1 = model_1.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
518/525 [============================>.] - ETA: 0s - loss: 0.0418
Epoch 00001: val_loss improved from inf to 0.01748, saving model to model\warehouse_checkpoint.h5py
INFO:tensorflow:Assets written to: model\warehouse_checkpoint.h5py\assets
525/525 [==============================] - 5s 10ms/step - loss: 0.0416 - val_loss: 0.0175
Epoch 2/50
519/525 [============================>.] - ETA: 0s - loss: 0.0159
Epoch 00002: val_loss improved from 0.01748 to 0.01303, saving model to model\warehouse_checkpoint.h5py
INFO:tensorflow:Assets written to: model\warehouse_checkpoint.h5py\assets
525/525 [==============================] - 4s 8ms/step - loss: 0.0159 - val_loss: 0.0130
Epoch 3/50
520/525 [============================>.] - ETA: 0s - loss: 0.0133
Epoch 00003: val_loss improved from 0.01303 to 0.01236, saving model to model\warehouse_checkpoint.h5py
INFO:tensorflow:Assets written to: model\warehouse_checkpoint.h5py\assets
525/525 [==============================] - 4s 7ms/step - los

In [27]:
warehouse_pred = model_1.predict(pred_feature)

In [32]:
warehouse_pred

array([[[0.00364342],
        [0.0001927 ],
        [0.00014389],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.00356236],
        [0.00017446],
        [0.00011333],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.00374717],
        [0.00018215],
        [0.00010831],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.00707334],
        [0.00062409],
        [0.0006882 ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.00567463],
        [0.00042275],
        [0.00041264],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.00481945],
        [0.00031406],
        [0.00026971],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]]], dtype=float32)

In [28]:
df = pd.concat([dangjin_fcst, energy['dangjin']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,dangjin
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [29]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['dangjin'] / c_dangjin
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [30]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [31]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16815, 24, 10), (4204, 24, 10), (4541, 24, 10), (672, 24, 10))

In [32]:
model_2 = Sequential()
model_2.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_2.add(Dropout(0.3))
model_2.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(loss='mean_squared_error', optimizer='adam')
model_2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 24, 24)            3360      
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 24)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 24)                4704      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [33]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[2], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_2 = model_2.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
523/526 [============================>.] - ETA: 0s - loss: 0.0333
Epoch 00001: val_loss improved from inf to 0.01762, saving model to model\dangjin_checkpoint.h5py
INFO:tensorflow:Assets written to: model\dangjin_checkpoint.h5py\assets
526/526 [==============================] - 12s 22ms/step - loss: 0.0333 - val_loss: 0.0176
Epoch 2/50
526/526 [==============================] - ETA: 0s - loss: 0.0128
Epoch 00002: val_loss improved from 0.01762 to 0.01051, saving model to model\dangjin_checkpoint.h5py
INFO:tensorflow:Assets written to: model\dangjin_checkpoint.h5py\assets
526/526 [==============================] - 11s 21ms/step - loss: 0.0128 - val_loss: 0.0105
Epoch 3/50
523/526 [============================>.] - ETA: 0s - loss: 0.0101
Epoch 00003: val_loss improved from 0.01051 to 0.00927, saving model to model\dangjin_checkpoint.h5py
INFO:tensorflow:Assets written to: model\dangjin_checkpoint.h5py\assets
526/526 [==============================] - 11s 21ms/step - loss: 0.01

In [34]:
dangjin_pred = model_2.predict(pred_feature)

In [35]:
dangjin_pred

array([[4.74926273e-05],
       [3.62857718e-05],
       [3.87181681e-05],
       [5.77434039e-05],
       [1.04945808e-04],
       [2.35617161e-04],
       [1.39677525e-03],
       [1.06867254e-02],
       [6.63819611e-02],
       [1.44523710e-01],
       [2.25134641e-01],
       [2.98468679e-01],
       [3.54885042e-01],
       [3.76739264e-01],
       [3.47859472e-01],
       [2.77808130e-01],
       [1.40040904e-01],
       [3.48936319e-02],
       [8.87754560e-03],
       [1.89480186e-03],
       [4.94092703e-04],
       [1.59353018e-04],
       [4.85157798e-05],
       [2.74654267e-05],
       [2.50613921e-05],
       [3.05692665e-05],
       [4.55169793e-05],
       [8.62549641e-05],
       [2.50160694e-04],
       [9.92089510e-04],
       [7.29590654e-03],
       [5.69003522e-02],
       [2.22143650e-01],
       [3.75492543e-01],
       [5.12767494e-01],
       [5.96117735e-01],
       [6.00310326e-01],
       [5.57518303e-01],
       [4.72746819e-01],
       [3.39121044e-01],


In [36]:
df = pd.concat([dangjin_fcst, energy['ulsan']], axis=1)
df
# df['dangjin_floating'].isnull().sum()

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,Wind_X,Wind_Y,Day sin,Day cos,Year sin,Year cos,declination,sunrise,sunset,altitude,Insolation,ulsan
0,2018-03-02 00:00:00,-2.000000,55.000000,6.700000,336.000000,1.0,6.120755,-2.725136,-7.071068e-01,-7.071068e-01,0.858280,0.513182,-8.293705,6.993442,17.189351,0.0,0.0,0.0
1,2018-03-02 01:00:00,-2.333333,55.000000,6.133333,337.000000,1.0,5.645763,-2.396484,-8.660254e-01,-5.000000e-01,0.858648,0.512566,-8.293705,6.993442,17.189351,0.0,0.0,0.0
2,2018-03-02 02:00:00,-2.666667,55.000000,5.566667,338.000000,1.0,5.161323,-2.085310,-9.659258e-01,-2.588190e-01,0.859015,0.511951,-8.293705,6.993442,17.189351,0.0,0.0,0.0
3,2018-03-02 03:00:00,-3.000000,55.000000,5.000000,339.000000,1.0,4.667902,-1.791840,-1.000000e+00,-3.848660e-12,0.859382,0.511335,-8.293705,6.993442,17.189351,0.0,0.0,0.0
4,2018-03-02 04:00:00,-3.333333,56.666667,4.700000,343.666667,1.0,4.510317,-1.321758,-9.659258e-01,2.588190e-01,0.859748,0.510719,-8.293705,6.993442,17.189351,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,5.333333,83.333333,7.466667,20.333333,4.0,7.001396,2.594527,5.000000e-01,-8.660254e-01,0.858846,0.512234,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26300,2021-03-01 20:00:00,4.666667,81.666667,7.333333,23.666667,4.0,6.716573,2.943710,2.588190e-01,-9.659258e-01,0.859213,0.511618,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26301,2021-03-01 21:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,4.907520e-12,-1.000000e+00,0.859579,0.511002,-8.293705,6.993442,17.189351,0.0,0.0,NaN
26302,2021-03-01 22:00:00,4.000000,80.000000,7.200000,27.000000,4.0,6.415247,3.268732,-2.588190e-01,-9.659258e-01,0.859945,0.510386,-8.293705,6.993442,17.189351,0.0,0.0,NaN


In [37]:
df.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
df_scaled = scaler.fit_transform(df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled['Generation'] = df['ulsan'] / c_ulsan
scale_cols.append('Generation')
df_scaled.columns = scale_cols

In [38]:
feature_cols = ['Temperature', 'Humidity', 'Cloud', 'Wind_X', 'Wind_Y', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Insolation']
label_cols = ['Generation']

window_size = 24
n = len(df_scaled)

train = df_scaled[:int(n*0.8)]
train.dropna(axis=0, inplace=True)
test = df_scaled[int(n*0.8):int(-(24*29))]
test.dropna(axis=0, inplace=True)
pred_input = df_scaled[int(-(24*29)):]

D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\data_engeneering\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [39]:
train_feature, train_label = train[feature_cols], train[label_cols]
train_feature, train_label = make_dataset(train_feature, train_label, window_size)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

test_feature, test_label = test[feature_cols], test[label_cols]
test_feature, test_label = make_dataset(test_feature, test_label, window_size)

pred_feature, pred_label = pred_input[feature_cols], pred_input[label_cols]
pred_feature, pred_label = make_dataset(pred_feature, pred_label, window_size)

x_train.shape, x_valid.shape, test_feature.shape, pred_feature.shape

((16815, 24, 10), (4204, 24, 10), (4541, 24, 10), (672, 24, 10))

In [40]:
model_3 = Sequential()
model_3.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=True)
          )

# model.add(Dense(1))
model_3.add(Dropout(0.3))
model_3.add(LSTM(24, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               return_sequences=False)
          )
model_3.add(Dense(1, activation='sigmoid'))
model_3.compile(loss='mean_squared_error', optimizer='adam')
model_3.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 24, 24)            3360      
_________________________________________________________________
dropout_4 (Dropout)          (None, 24, 24)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 24)                4704      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25        
Total params: 8,089
Trainable params: 8,089
Non-trainable params: 0
_________________________________________________________________


In [41]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint(filename[3], monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history_3 = model_3.fit(x_train, y_train, epochs=epoch,
                                      validation_data=(x_valid, y_valid), 
                                      callbacks=[early_stop, checkpoint])

Epoch 1/50
523/526 [============================>.] - ETA: 0s - loss: 0.0346
Epoch 00001: val_loss improved from inf to 0.01596, saving model to model\ulsan_checkpoint.h5py
INFO:tensorflow:Assets written to: model\ulsan_checkpoint.h5py\assets
526/526 [==============================] - 12s 22ms/step - loss: 0.0345 - val_loss: 0.0160
Epoch 2/50
525/526 [============================>.] - ETA: 0s - loss: 0.0143
Epoch 00002: val_loss improved from 0.01596 to 0.01235, saving model to model\ulsan_checkpoint.h5py
INFO:tensorflow:Assets written to: model\ulsan_checkpoint.h5py\assets
526/526 [==============================] - 11s 21ms/step - loss: 0.0143 - val_loss: 0.0123
Epoch 3/50
523/526 [============================>.] - ETA: 0s - loss: 0.0120
Epoch 00003: val_loss improved from 0.01235 to 0.01127, saving model to model\ulsan_checkpoint.h5py
INFO:tensorflow:Assets written to: model\ulsan_checkpoint.h5py\assets
526/526 [==============================] - 11s 21ms/step - loss: 0.0120 - val_los

In [42]:
ulsan_pred = model_3.predict(pred_feature)

In [43]:
ulsan_pred

array([[5.06469914e-05],
       [3.58063007e-05],
       [3.44955188e-05],
       [4.16782459e-05],
       [6.23976448e-05],
       [1.53720379e-04],
       [7.13288784e-04],
       [7.99226761e-03],
       [7.05473721e-02],
       [1.79855466e-01],
       [3.08252484e-01],
       [4.35182303e-01],
       [4.53088939e-01],
       [4.46262002e-01],
       [4.12548542e-01],
       [2.92074442e-01],
       [9.00746584e-02],
       [1.37466192e-02],
       [6.14205003e-03],
       [2.47889757e-03],
       [9.96649265e-04],
       [4.20749187e-04],
       [2.01433897e-04],
       [1.15228657e-04],
       [8.04355877e-05],
       [6.54297910e-05],
       [7.06373903e-05],
       [9.26452703e-05],
       [1.55240297e-04],
       [6.13838434e-04],
       [3.79627943e-03],
       [4.29430902e-02],
       [2.83587933e-01],
       [4.58642423e-01],
       [5.49923837e-01],
       [5.82184494e-01],
       [5.77663064e-01],
       [5.27020156e-01],
       [4.74732518e-01],
       [4.07469630e-01],


In [ ]:
day = 50
day_range = 10

plt.figure(figsize=(20, 5))
plt.title('Dangjin', fontsize=20)
plt.plot(test_label[-day*24:-(day-day_range)*24], label = 'actual')
plt.plot(pred[-day*24:-(day-day_range)*24], label = 'prediction')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(floating_pred * c_floating, label = 'dangjin_floating_pred')
plt.plot(warehouse_pred * c_warehouse, label = 'dangjin_warehouse_pred')
plt.plot(dangjin_pred * c_dangjin, label = 'dangjin_pred')
plt.plot(ulsan_pred * c_ulsan, label = 'ulsan_pred')
plt.legend()
plt.show()

In [45]:
submission = pd.read_csv('data/sample_submission.csv')

In [46]:
submission.iloc[:24*28, 1] = floating_pred * c_floating
submission.iloc[:24*28, 2] = warehouse_pred * c_warehouse
submission.iloc[:24*28, 3] = dangjin_pred * c_dangjin
submission.iloc[:24*28, 4] = ulsan_pred * c_ulsan

In [47]:
# submission.to_csv('data/submission(lstm 조정).csv', index=False)
# submission.head(48)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.084255,1.024101e-06,0.047493,0.025323
1,2021-02-01 02:00:00,0.084378,1.000578e-07,0.036286,0.017903
2,2021-02-01 03:00:00,0.094282,6.128192e-08,0.038718,0.017248
3,2021-02-01 04:00:00,0.126243,4.806286e-08,0.057743,0.020839
4,2021-02-01 05:00:00,0.233382,1.258633e-07,0.104946,0.031199
5,2021-02-01 06:00:00,0.552535,6.315594e-06,0.235617,0.076860
6,2021-02-01 07:00:00,1.877517,2.757749e-03,1.396775,0.356644
7,2021-02-01 08:00:00,8.623838,6.786704e-01,10.686726,3.996134
8,2021-02-01 09:00:00,47.421368,1.795155e+01,66.381958,35.273685
9,2021-02-01 10:00:00,103.984924,7.016384e+01,144.523712,89.927734
